In [1]:
import os
import pandas as pd
import numpy as np
import PIL
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import load_model,save_model
from keras.callbacks import ModelCheckpoint
from matplotlib.patches import Rectangle
import os
from scipy.misc import imsave

import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from math import sqrt
from keras.callbacks import History 
import gc

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing import image

def read_img(filepath, size):
    img = image.load_img((filepath), target_size=size)
    img = image.img_to_array(img,data_format='channels_last')
    return img

os.chdir('/home/rsk/Documents/Projects/YOLO/25x25/')

X = []

for i in range(8000):
    img = read_img(str(i)+'.png',(25,25))
    X.append(img)

In [3]:


truth = pd.read_csv('25x25_truth.csv')
y = np.array(truth)
y = np.array(y[:,1:])


X= np.array(X)
y = np.array(y)


train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size=25


trainX = X[:7000,:,:,:]
trainy = y[:7000,:]

testX = X[7000:,:,:,:]
testy = y[7000:,:]

train_generator = train_datagen.flow(x=trainX,y=trainy,batch_size=batch_size)
test_generator = test_datagen.flow(x=testX,y=testy,batch_size=batch_size)

In [4]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou


def getIoU(model,testData,testTruth,rescale=True):
    
    sample = testData
    if rescale==True:
        sample = testData/255.
    pred = model.predict(sample)
    
    
    testTruth = [[i[0],i[1],i[0]+i[2],i[1]+i[3]] for i in testTruth]
    pred = [[i[0],i[1],i[0]+i[2],i[1]+i[3]] for i in pred]
    
    IoUSum = 0.0
    for i in range(len(testTruth)):
        IoUSum += bb_intersection_over_union(testTruth[i],pred[i])
    
    meanIoU = IoUSum/len(testTruth)
    
    return(meanIoU)

def getMSE(model,testData,testTruth,rescale=True):
    
    sample = testData
    if rescale==True:
        sample = sample/255.
    pred = model.predict(sample)
    
    MSESum = 0.0
    for i in range(len(testTruth)):
        a = np.array(testTruth[i])
        b = pred[i]
        MSESum += sqrt(np.sum((a-b)**2))
    
    meanMSE = MSESum/len(testTruth)
    
    return(meanMSE)
   

In [22]:
def getSampleIOU(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2] + boxA[0], boxB[2] + boxB[0])
	yB = min(boxA[3] + boxA[1], boxB[3] + boxB[1])
 
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2]  + 1) * (boxA[3]  + 1)
	boxBArea = (boxB[2]  + 1) * (boxB[3]  + 1)
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

In [6]:
import keras.backend as K

def iou(y_true,y_pred):
    """
    IOU loss function for a single sample. Cannot be used for batch
    """
    a=y_true[0]
    b=y_pred[0]
    
    
    xa = K.maximum(a[0],b[0])
    ya = K.maximum(a[1],b[1])
    xb = K.minimum(a[0]+a[2],b[0]+b[2])
    yb = K.minimum(a[1]+a[3],b[1]+b[3])
    
    
    interArea = K.maximum(K.constant(0.0), xb -xa + K.constant(1.0))*K.maximum(K.constant(0.0), yb -ya + K.constant(1.0))
    
    boxAarea= (a[2]  + 1) * (a[3]  + 1)
    boxBarea= (b[2]  + 1) * (b[3]  + 1)
    
    iou = interArea / (boxAarea + boxBarea - interArea)
    
    return(iou)

def iou2(y_true,y_pred):
    """
    IOU loss function that can handle batches of sample
    """
    a=y_true
    b=y_pred
    
    xa = K.maximum(a[:,0],b[:,0])
    ya = K.maximum(a[:,1],b[:,1])
    xb = K.minimum(a[:,0]+a[:,2],b[:,0]+b[:,2])
    yb = K.minimum(a[:,1]+a[:,3],b[:,1]+b[:,3])
    
    
    interArea = K.maximum(K.constant(0.0), xb -xa + K.constant(1.0))*K.maximum(K.constant(0.0), yb -ya + K.constant(1.0))
    
    boxAarea= (a[:,2]  + 1) * (a[:,3]  + 1)
    boxBarea= (b[:,2]  + 1) * (b[:,3]  + 1)
    
    iou = interArea / (boxAarea + boxBarea - interArea)
    return(-1*K.mean(iou))
    

In [7]:
def cleanFolder(buildNo,keep=5):
    
    os.chdir('./Models')
    fileList = os.listdir('./')
    
    selectFileList = []
    for filename in fileList:
        if str(buildNo) in filename.split("_"):
            selectFileList.append(filename)
            
    a = []
    for i in selectFileList:
        val = float(i[:-5].split("_")[3])
        a.append([val,i])
        
    a = sorted(a,key=lambda x : x[0])
    
    keep = min(keep,len(a))
    
    keepList = [i[1] for i in a[:keep]]
    dropList = [i[1] for i in a[keep:]]
    
    for filename in dropList:
        os.remove(filename)
            
    return(keepList)
 

In [8]:
def runModel(model,epochs,build=1,optimizer = 'adagrad',keep=5):
    
        os.chdir('/home/rsk/Documents/Projects/YOLO/BoundingBox/')
    
        model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['mse',iou2])
        hist = History()
        checkPoint = ModelCheckpoint(filepath='./Models/model_'+str(build)+'_{epoch:02d}_{val_loss:.2f}.hdf5',verbose=1,save_best_only=True,mode=min)
    
        model.fit_generator(train_generator,epochs=epochs,validation_data=test_generator,callbacks = [hist,checkPoint],verbose=1)
        
        keepList = cleanFolder(build,keep=keep)
        
        for filename in keepList:
            model = load_model(filename)
            epoch = filename.split("_")[2]
            mse = filename.split("_")[3]
            IoU = round(getIoU(model,testX,testy),3)
            print("Epoch : {} MSE : {} IoU : {}".format(epoch,mse,IoU))
        
        del model
        gc.collect()
            
        

### Ensembling for single predictions

In [23]:
def ensemblePrediction(obs,modelList):
    """
    Running ensemble for a single sample
    """
    
    os.chdir('/home/rsk/Documents/Projects/YOLO/BoundingBox/Models')
    
    sample = testX[obs]
    
    sample = sample.reshape((1,25,25,3))
    sample = sample/255.
    truth = [testy[obs]]
    
    predList=[]
    iouList = []
    
    for model in modelList:
        model = load_model(model)
        pred = model.predict(sample)
        predList.append(pred[0])
        iouList.append(round(getIoU(model, sample, truth,rescale=False),3))
        
        
        
    predList = np.array(predList)
    finalPred = [ np.mean(predList[:,0]) , np.mean(predList[:,1]) , np.mean(predList[:,2]) , np.mean(predList[:,3])  ]
    finalIOU = getSampleIOU(truth[0],finalPred)
             
    print(predList)
    
    print("Individual model IOU's : {}".format(iouList))
    print("Ensemble model IOU : {}".format(finalIOU))
    return(finalPred)
        
    
        
    

In [24]:
modelList = [
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_84_0.04.hdf5',
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_124_0.03.hdf5',
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_142_0.03.hdf5',
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_143_0.03.hdf5']
a=ensemblePrediction(122,modelList)

[[ 2.7369304  4.672687  10.420157   6.518721 ]
 [ 3.0074704  4.6710057  9.930847   6.2390823]
 [ 2.9920006  4.8616185 10.05343    6.034426 ]
 [ 2.8999557  4.766427  10.155827   6.2567735]]
Individual model IOU's : [0.897, 0.937, 0.961, 0.951]
Ensemble model IOU : 0.951769475324


### Running Ensemble for entire Test data

In [31]:
def ensemblePredictionBatch(X,y,modelList):
    """
    Running ensemble predictions for an entire batch of images 
    """
    
    os.chdir('/home/rsk/Documents/Projects/YOLO/BoundingBox/Models')
    
    sample = X
    
#     sample = sample.reshape((1,25,25,3))
    sample = sample/255.
    truth = y
    
    predList=[]
    iouList = []
    
    for model in modelList:
        model = load_model(model)
        pred = model.predict(sample)
        predList.append(pred)
        iouList.append(getIoU(model,sample,truth,rescale = False))
#         iouList.append(round(getIoU(model, sample, truth,rescale=False),3))
    
        
        
    predList = np.array(predList)
    finalPred = np.mean(predList,axis=0)
    
    iou=[]
    for i in range(len(truth)):
        sampleIou = getSampleIOU(truth[i],finalPred[i])
        iou.append(sampleIou)
    finalIOU = np.mean(iou)
    
    
    print("Individual model IOU's : {}".format(iouList))
    print("Ensemble model IOU : {}".format(finalIOU))
    return(finalPred)
        
    
        
    

In [32]:
modelList = [
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_142_0.03.hdf5',
'/home/rsk/Documents/Projects/YOLO/BoundingBox/Models/model_3_143_0.03.hdf5']
a = ensemblePredictionBatch(testX,testy,modelList)

Individual model IOU's : [0.915626451977324, 0.9120252077245947]
Ensemble model IOU : 0.923542096116
